In [175]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime

In [176]:
def selecionar_aprovados(dados):

    df = dados.copy(deep=True)

    # Seleciona os 81 primeiros candidatos aprovados
    df.loc[df.loc[df["APROVADO DISCURSIVA"] == True].head(81).index, 'SITUAÇÃO PARCIAL'] = "Aprovado nas vagas de ampla concorrência"

    # Seleciona os candidatos aprovados PCD que ainda não foram incluídos
     
    df.loc[df.loc[(df['SITUAÇÃO PARCIAL'].isna()) & 
                  (df["SITUACAO OBJETIVA"] == "Aprovado PcD") &
                  (df["APROVADO DISCURSIVA"] == True)].head(9).index, 'SITUAÇÃO PARCIAL'] = "Aprovado nas vagas de cota PCD"

    # Caso não haja 9 candidatos aprovados PCD, completa com aprovados na Ampla
    if len(df[df['SITUAÇÃO PARCIAL'] == "Cota PCD"]) < 9:
        df.loc[df.loc[(df['SITUAÇÃO PARCIAL'].isna()) &
                      (df["APROVADO DISCURSIVA"] == True)].head(9 - len(df[df['SITUAÇÃO PARCIAL'] == "Aprovado nas vagas de cota PCD"])).index, 'SITUAÇÃO PARCIAL'] = "Aprovado nas vagas de ampla concorrência"

    df.loc[(df['SITUAÇÃO PARCIAL'].isna()) & 
           (df["APROVADO DISCURSIVA"] == True), 'SITUAÇÃO PARCIAL'] = "Cadastro de Reservas"

    df['SITUAÇÃO PARCIAL'] = df['SITUAÇÃO PARCIAL'].fillna('Reprovado na discursiva')
      
    # Retorna os 81 primeiros aprovados + os 9 primeiros aprovados PCD
    return df

In [177]:
resultado_objetiva = pd.read_csv('./data/resultado-objetiva.csv', sep=";", index_col=['INSC_NUMB', 'NOME'])
resultado_discursiva = pd.read_csv('./data/resultado-discursiva.csv', sep=";", index_col=['INSC_NUMB', 'NOME'])

In [178]:
resultado_preliminar = resultado_objetiva.merge(resultado_discursiva, on=['INSC_NUMB', 'NOME'], how='left')

In [179]:
resultado_preliminar['TOTAL'] = resultado_preliminar['TOTAL OBJETIVA'] + resultado_preliminar['TOTAL DISCURSIVA'].fillna(value=0)
resultado_preliminar['APROVADO DISCURSIVA'] = resultado_preliminar['TOTAL DISCURSIVA'] >= 40
resultado_preliminar['NASC'] = pd.to_datetime(resultado_preliminar['NASC'], format='%d/%m/%Y')
today = datetime.today()
resultado_preliminar['IDADE'] = (today - resultado_preliminar['NASC']).astype('timedelta64[Y]')
resultado_preliminar['IDOSO'] = resultado_preliminar['IDADE'] >= 60

In [180]:
resultado_preliminar_ordenado = resultado_preliminar.sort_values(by=['APROVADO DISCURSIVA', 'TOTAL', 'IDOSO', 'TOTAL DISCURSIVA', 'P2', 'CD', 'DEV', 'BD', 'NASC'], 
                                               ascending=[False, False, False, False, False, False, False, False, True])

In [181]:
resultado_preliminar_ordenado = resultado_preliminar_ordenado.reset_index(drop=False)
resultado_preliminar_ordenado['POS'] = resultado_preliminar_ordenado.index + 1

In [182]:
resultado_preliminar_ordenado[resultado_preliminar_ordenado["SITUACAO OBJETIVA"] == "Aprovado PcD"]

,INSC_NUMB,NOME,POS OBEJETIVA,NASC,PT,RLM,DALE,DCDH,DCDE,DP,...,Q1,Q2,Q3,Q4,TOTAL DISCURSIVA,TOTAL,APROVADO DISCURSIVA,IDADE,IDOSO,POS
40,266004294,Evandro Lima Nascimento,434,1967-01-03,5,2,6,6,8,3,...,15.9,17.9,19.4,18.8,72.0,167.0,True,56.0,False,41
86,266005430,Lucas Araújo Borges,205,1994-11-29,4,7,9,4,5,4,...,1.1,17.8,17.9,14.3,51.1,158.1,True,28.0,False,87
189,266004199,Sérgio Guimarães Villaça,318,1979-06-08,5,6,6,3,7,3,...,9.1,16.4,7.0,12.2,44.7,145.7,True,43.0,False,190
197,266011902,Joao Ricardo Cogo Zanetti,327,1984-06-24,5,4,6,4,4,2,...,16.3,15.3,5.0,8.0,44.6,144.6,True,38.0,False,198
204,266007672,Flávio José Gonçalves Facchinetti,290,1979-05-24,3,5,7,6,4,3,...,11.4,10.3,11.1,8.9,41.7,143.7,True,43.0,False,205
266,266014965,Cosme Diego Da Silva Augusto,479,1990-06-16,4,3,8,6,3,1,...,13.5,8.3,19.6,0.7,42.1,135.1,True,32.0,False,267
278,266024969,Diogo Majela Correa Lopes,464,1994-03-02,4,1,6,2,5,2,...,3.8,19.7,14.2,3.0,40.7,133.7,True,29.0,False,279
334,266025052,José Antonio Dos Santos Barbosa,46,1981-12-21,4,6,12,4,7,4,...,NaN,NaN,NaN,NaN,NaN,123.0,False,41.0,False,335
338,266009828,Marcelo Cicco Do Nascimento,51,1984-09-03,4,5,8,7,6,2,...,NaN,NaN,NaN,NaN,NaN,121.0,False,38.0,False,339
367,266015617,Moises De Morais Henriques,101,1990-10-16,2,5,10,5,4,5,...,NaN,NaN,NaN,NaN,NaN,115.0,False,32.0,False,368


In [183]:
resultado_preliminar_ordenado = selecionar_aprovados(resultado_preliminar_ordenado)

In [184]:
resultado_preliminar_ordenado[['POS', 'INSC_NUMB', 'NOME', 'TOTAL', 'IDOSO', 'TOTAL DISCURSIVA', 'P2', 'CD','DEV', 'BD', 'NASC', 'VAGAS']]

KeyError: "['VAGAS'] not in index"